In [50]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [51]:
from langchain.text_splitter import TokenTextSplitter

In [52]:
with open("/Users/sonhunseo/Desktop/공모전/4. 경기도_GenAI·공공데이터_창업경진대회/similarity.txt") as f:
    context = f.read()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(context)

embeddings = OpenAIEmbeddings()

In [53]:
texts[0]

'계약한직후에 벌어질 수 있는 전세사기,"1. 전세계약 당일 임대인 변경 및 주택담보대출 실행\n사례 : 2021년 4월, A씨는 서울 동작구 상도동의 한 주택에 입주하게 되었어요. 근저당이 없어 권리관계가 깨끗한 집이었죠. A씨는 전입신고와 동시에 확정일자까지 받았 어요. 이쯤 되면 세입자로서 할 수 있는 모든 일을 다 했다고 볼 수 있지요. 그런데 어느 날, 법원으로부터 고지서 한 통을 받게 되었어요. 집이 경매에 넘어가게 되었으니 퇴거해달라는 내용이었죠. 어떻게 된 일일까요? 알고 보니 전입신고 당일, 임대인 B씨가 자산이 거의 없다시피 한 C씨에게 소유권을 넘겼고, C씨는 주택담보대출을 받았어요. 그러다 대출을 갚을 여력이 되지 않았던 C씨는 잠적했고, 집은 경매에 넘어가게 된 거죠."\n계약한직후에 벌어질 수 있는 전세사기,"2. 이중계약\n사례 : 한 청년 A는 마음에 드는 전세 물건을 찾아 권리관계를 확인하고 임대인을 직접 만나 계약을 체결하였어요. 그러�'

In [54]:
docsearch = FAISS.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

In [55]:
query = "계약을 하려는데 집주인말고 다른사람이 왔어"
docs = docsearch.similarity_search(query)[0:2]

In [56]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [57]:
template = """
질문자의 상황 : {question}
=========
{summaries}
=========
질문자는 현재 본인이 전세사기의 위험이 있는지 알고싶어해.
너는 위 사례와 질문자의 상황의 비슷한점을 판단해서 어떠한점이 위험한지 알려줘야해
너는 반드시 답변시 질문자의 상황과 사례를 비교하면서 어떤점이 위험한지 알려줘야해
"""

PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0, max_tokens=1500), chain_type="stuff", prompt=PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


{'output_text': '\n질문자의 상황과 사례를 비교해보면, 질문자가 계약을 하려고 할 때 다른 사람이 왔다는 사실이 위험한 지표가 될 수 있습니다. 이는 임대인이 하나의 주택을 대상으로 두 사람 이상과 임대차계약을 체결하는 것이 불법이기 때문입니다. 또한, 임대인이 실제 계약하고자 하는 내용과 다른 계약 방식을 제안하는 것도 위험합니다. 또한, 가짜 임대인과의 계약과 신탁회사의 동의 없는 계약을 하는 것도 위험합니다. 따라서 질문자는 계약하기 전에 임대인의 신원을 확인하고, 실제 계약하고자 하는 내용과 다른 계약 방식을 제안하는 것을 거부해야 합니다.'}